# **Assignment 2 – Information Visualisation**
## Data Processing for CSO High Value Dataset (EAA17 Greenhouse Gas Emissions)

## Goal
The goal is to perform data manipulation outside of the Vega-Lite environment, using Python. Making clear what transformations / filtering I perform and making the processed data set available.

## Dictionary
"STATISTIC": Internal CSO identifier for the type of emission being measured.

"Statistic Label": Human-readable name of the emission type being measured.

"TLIST(A1)": Time period identifier, stores the year as a string in YYYY format (matches Year).

"Year": The calendar year of the measurement.

"C04273V05050": NACE sector classification code.

"NACE Rev. 2 Sector": Full name of the economic sector following NACE Rev. 2 classification. Includes sector code in parentheses.

"UNIT": Unit of measurement for the VALUE field.

"VALUE": Actual emissions measurement, the quantity of the specified emission type from the specified sector in the specified year, measured in thousand tonnes (or thousand tonnes of CO2 equivalent for greenhouse gases).

# **1. Import libraries**

I import the necessary packages and set up everything I need.

In [82]:
import pandas as pd
import numpy as np

# Display settings for clarity
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

I then load the original downloaded CSO dataset saved as `eaa17_original.csv`, containing 1 record per property. Showing the number of rows and columns in dataset, and the first 5 rows for preview

In [83]:
# Load original downloaded CSO dataset
df_raw = pd.read_csv("eaa17_original.csv")

# Inspect structure
print("Shape:", df_raw.shape)
df_raw.head(5)

Shape: (10650, 8)


,STATISTIC,Statistic Label,TLIST(A1),Year,C04273V05050,NACE Rev. 2 Sector,UNIT,VALUE
0,EAA17C01,Carbon dioxide (CO2),2009,2009,01,"Crop and animal production, hunting and relate...",Thousand Tonnes,"1,153.40"
1,EAA17C01,Carbon dioxide (CO2),2009,2009,02,Forestry and logging (02),Thousand Tonnes,53.67
2,EAA17C01,Carbon dioxide (CO2),2009,2009,03,Fishing and aquaculture (03),Thousand Tonnes,119.67
3,EAA17C01,Carbon dioxide (CO2),2009,2009,0509,Mining and quarrying (05-09),Thousand Tonnes,58.15
4,EAA17C01,Carbon dioxide (CO2),2009,2009,1012,"Manufacture of food products, beverages and to...",Thousand Tonnes,929.04


# **2. Overview and Checks**
I need to display column names, data types, and number of non-missing values for each column. To better understand which columns have missing data, which are numeric or categorical, if data types are correct, and if there’s missing or inconsistent data.

In [84]:
df_raw.info()

print('\n Analysis numerical categories\n')
print(df_raw[['TLIST(A1)','Year','VALUE']].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10650 entries, 0 to 10649
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   STATISTIC           10650 non-null  object 
 1   Statistic Label     10650 non-null  object 
 2   TLIST(A1)           10650 non-null  int64  
 3   Year                10650 non-null  int64  
 4   C04273V05050        10650 non-null  object 
 5   NACE Rev. 2 Sector  10650 non-null  object 
 6   UNIT                10650 non-null  object 
 7   VALUE               10650 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 665.8+ KB

 Analysis numerical categories

       TLIST(A1)      Year      VALUE
count  10,650.00 10,650.00  10,650.00
mean    2,016.00  2,016.00   3,011.25
std         4.32      4.32  34,864.76
min     2,009.00  2,009.00       0.00
25%     2,012.00  2,012.00       0.01
50%     2,016.00  2,016.00       0.78
75%     2,020.00  2,020.00      22.28
m

The dataset contains greenhouse gas emissions broken down by year, sector, and gas type. For the purposes of this visualisation, emissions are analysed in kilotonnes of CO₂ equivalent.

# **3. Data Pre-Processing**

## 3.1 Handle Missing and Invalid Values
First of all, I replace placeholder text for missing data ('???', 'na', ...) with actual NaN values that pandas can recognise as missing. 

After that, I can count missing values in each column. Sort them from most to least missing to see which categories might be more problematic.

In [85]:
df_raw.replace(['???', 'N/A', 'na', 'NaN', 'nan', 'None', ''], np.nan, inplace=True)

missing_list = df_raw.isna().sum().sort_values(ascending=False)
print('Missing values by category:\n', missing_list)

Missing values by category:
 STATISTIC             0
Statistic Label       0
TLIST(A1)             0
Year                  0
C04273V05050          0
NACE Rev. 2 Sector    0
UNIT                  0
VALUE                 0
dtype: int64


## 3.2 Standardise Data Types
Then I drop the column `TLIST(A1)` as I already have all the information I need separated in `Year`. And the columns `STATISTIC` and `C04273V05050` as I already have all the information I need separated in `Statistic Label` and `NACE Rev. 2 Sector` respectively.

And I drop duplicates from the complete dataset (which could be scraped twice or repeated).

In [86]:
# Drop columns
df_raw.drop(columns=['TLIST(A1)'], inplace=True)
df_raw.drop(columns=['STATISTIC'], inplace=True)
df_raw.drop(columns=['C04273V05050'], inplace=True)

# Drop duplicates (just in case)
df_raw.drop_duplicates(inplace=True)
# Print data types to confirm correct formats
print(df_raw.dtypes)
print('\nRows after dropping duplicates:', len(df_raw))

Statistic Label        object
Year                    int64
NACE Rev. 2 Sector     object
UNIT                   object
VALUE                 float64
dtype: object

Rows after dropping duplicates: 10650


**Notes:**

- No duplicates were present in the dataset, as the number of rows stayed the same as before dropping them.

In [87]:
# Get unique values in the UNIT column
unique_units = df_raw['UNIT'].unique()

print("Unique values in UNIT column:")
print(unique_units)
print("\nValue counts:")
print(df_raw['UNIT'].value_counts())

Unique values in UNIT column:
['Thousand Tonnes' 'Tonnes']

Value counts:
UNIT
Thousand Tonnes    8520
Tonnes             2130
Name: count, dtype: int64


In [88]:
# Rename columns for clarity and easier Vega-Lite encoding
df = df_raw.rename(columns={
    "NACE Rev. 2 Sector": "NACE Sector",
    "VALUE": "Value",
    "UNIT": "Unit"
})

df.sample(10)

,Statistic Label,Year,NACE Sector,Unit,Value
1692,Carbon dioxide from biomass (CO2_BIO),2017,Education (85),Thousand Tonnes,15.56
7146,Nitrous oxide in CO2 equivalent (N2O_CO2E),2019,Activities auxiliary to financial services and...,Thousand Tonnes,0.13
5264,Methane in CO2 equivalent (CH4_CO2E),2023,Manufacture of chemicals and chemical products...,Thousand Tonnes,0.06
7333,Nitrous oxide in CO2 equivalent (N2O_CO2E),2022,Manufacture of other transport equipment (30),Thousand Tonnes,0.01
5704,Nitrous oxide (N2O),2014,"Water collection, treatment and supply (36)",Tonnes,0.19
1987,Carbon dioxide from biomass (CO2_BIO),2021,Activities of extraterritorial organisations a...,Thousand Tonnes,0.00
2154,Hydrofluorocarbons in CO2 equivalent (HFC_CO2E),2009,"Water collection, treatment and supply (36)",Thousand Tonnes,0.21
7671,Perfluorocarbons in CO2 equivalent (PFC_CO2E),2012,Mining and quarrying (05-09),Thousand Tonnes,0.00
8686,Sulphur hexafluoride and nitrogen trifluoride ...,2011,"Water collection, treatment and supply (36)",Thousand Tonnes,0.00
5468,Nitrous oxide (N2O),2011,Forestry and logging (02),Tonnes,11.33


In [89]:
# Ensure emissions are numeric
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

df = df.dropna(subset=["Value"])

print("After cleaning missing values:", df.shape)

After cleaning missing values: (10650, 5)


## 3.3 Summary Statistics
Now I want to know the statistical summaries (count, mean, std, min, 25%, 50%, 75%, max, median) for numerical categories.

In [90]:
print("Final processed dataset shape:", df.shape)
df.describe()

Final processed dataset shape: (10650, 5)


,Year,Value
count,"10,650.00","10,650.00"
mean,"2,016.00","3,011.25"
std,4.32,"34,864.76"
min,"2,009.00",0.00
25%,"2,012.00",0.01
50%,"2,016.00",0.78
75%,"2,020.00",22.28
max,"2,023.00","582,909.01"


# **4. Save the CSV**
Finally, I write the final cleaned data to a new CSV file.

In [91]:
# Export processed dataset for Vega-Lite dashboard
df.to_csv("eaa17_processed.csv", index=False)

print("Processed dataset saved as eaa17_processed.csv")

Processed dataset saved as eaa17_processed.csv
